In [ ]:
%%capture
# Unsloth kütüphanesini kuruyoruz (Eğitimi 2 kat hızlandırır)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes


In [7]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Modelin hafızasında tutacağı kelime uzunluğu
dtype = None
load_in_4bit = True # Bellek tasarrufu için 4-bit sıkıştırma

# Meta'nın Llama-3-8B modelini Unsloth üzerinden çekiyoruz
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Modeli eğitime (LoRA - Low Rank Adaptation) hazırlama
# Bu teknik, modelin tamamını değil sadece %1'lik kısmını eğiterek aynı sonucu almamızı sağlar.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
)

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [13]:
from datasets import load_dataset
import json
import random

print("🛠️ Dış kaynaklara erişilemedi. Mühendislik çözümü: Sentetik Veri Üretiliyor...")

# --- 1. ADIM: Sentetik Veri Oluşturucu ---
# Bu kısım, farklı şablonları birleştirerek binlerce farklı Türkçe soru-cevap üretir.

templates = [
    # Şablon 1: Başkent Soruları
    {
        "func": lambda x: f"{x[0]}'nın başkenti neresidir?",
        "input": "",
        "ans": lambda x: f"{x[0]}'nın başkenti {x[1]}'dir.",
        "data": [("Türkiye", "Ankara"), ("Fransa", "Paris"), ("Almanya", "Berlin"), ("İtalya", "Roma"), ("İngiltere", "Londra"), ("Japonya", "Tokyo"), ("Rusya", "Moskova"), ("İspanya", "Madrid")]
    },
    # Şablon 2: Matematik (Toplama)
    {
        "func": lambda x: f"{x[0]} ile {x[1]} sayılarının toplamı kaçtır?",
        "input": "Matematik işlemi",
        "ans": lambda x: f"{x[0]} + {x[1]} = {x[0] + x[1]} eder.",
        "data": [(random.randint(1, 100), random.randint(1, 100)) for _ in range(500)] # 500 matematik sorusu
    },
    # Şablon 3: Yazılım Tanımları
    {
        "func": lambda x: f"{x[0]} nedir?",
        "input": "Yazılım Kavramları",
        "ans": lambda x: x[1],
        "data": [
            ("Python", "Python, okunabilirliği yüksek, nesne yönelimli ve yorumlanan bir programlama dilidir."),
            ("Yapay Zeka", "Yapay zeka, insan zekasını taklit eden ve verilerden öğrenen sistemlerdir."),
            ("Docker", "Docker, uygulamaları konteynerler içinde izole bir şekilde çalıştırmayı sağlayan bir platformdur."),
            ("API", "API (Application Programming Interface), iki yazılımın birbiriyle konuşmasını sağlayan arayüzdür."),
            ("Loop", "Loop (Döngü), bir kod bloğunun belirli bir koşul sağlanana kadar tekrar etmesidir.")
        ]
    },
    # Şablon 4: Çarpma İşlemi (Modelin mantığını geliştirmek için)
    {
        "func": lambda x: f"{x[0]} kere {x[1]} kaç eder?",
        "input": "",
        "ans": lambda x: f"Sonuç {x[0] * x[1]} eder.",
        "data": [(random.randint(2, 20), random.randint(2, 20)) for _ in range(500)]
    }
]

synthetic_data = []

# Veriyi çoğaltma ve karıştırma döngüsü
# Az sayıda şablonla çok sayıda veri üretiyoruz
target_count = 2000 # 2000 satırlık veri üreteceğiz

while len(synthetic_data) < target_count:
    for temp in templates:
        for item in temp["data"]:
            entry = {
                "instruction": temp["func"](item),
                "input": temp["input"],
                "output": temp["ans"](item)
            }
            synthetic_data.append(entry)

# Hedef sayıya ulaştıysak fazlasını kırp, azsa karıştırıp ekle
random.shuffle(synthetic_data)
synthetic_data = synthetic_data[:target_count]

# JSON olarak kaydet
with open("synthetic_alpaca_tr.json", "w", encoding="utf-8") as f:
    json.dump(synthetic_data, f, ensure_ascii=False)

print(f"Başarıyla {len(synthetic_data)} adet Türkçe eğitim verisi üretildi ve kaydedildi.")

# --- 2. ADIM: Veriyi Yükle ve Formatla ---
dataset = load_dataset("json", data_files="synthetic_alpaca_tr.json", split="train")

alpaca_prompt = """Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
{}

### Girdi:
{}

### Yanıt:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
print("Veri seti eğitime hazır.")

🛠️ Dış kaynaklara erişilemedi. Mühendislik çözümü: Sentetik Veri Üretiliyor...
Başarıyla 2000 adet Türkçe eğitim verisi üretildi ve kaydedildi.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Veri seti eğitime hazır.


In [17]:
import psutil
import builtins

# --- KRİTİK DÜZELTME ---
# Kütüphanenin içine psutil'i zorla enjekte ediyoruz.
# Bu sayede "NameError" hatasını bypass edeceğiz.
builtins.psutil = psutil
# -----------------------

from trl import SFTTrainer
from transformers import TrainingArguments
import torch

# dataset_num_proc parametresini açıkça belirtiyoruz
# psutil hatası almamak için işlemci sayısını elle veriyoruz.

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Hata almamak için loglamayı kapattık
    ),
)

print("Eğitim Başlıyor.")
trainer_stats = trainer.train()
print("Eğitim Başarıyla Tamamlandı.")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Eğitim Başlıyor.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 2 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,3.153900
2,3.172700
3,2.922200
4,2.647500
5,2.193400
6,1.673800
7,1.067300
8,0.718600
9,0.572700
10,0.508700


Eğitim Başarıyla Tamamlandı.


In [18]:
FastLanguageModel.for_inference(model) # Modeli soru-cevap moduna al

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Yapay zeka mühendisliği nedir?", # Sorunu buraya yaz
        "",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
# Çıktıyı temizle ve yazdır
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("### Yanıt:")[-1].strip())

Yapay zeka mühendisliği, insan zekasını taklit eden sistemlerin geliştirilmesidir.


In [19]:
# --- 5. ADIM: TEST (INFERENCE) ---
from transformers import TextStreamer

FastLanguageModel.for_inference(model) # Modeli "Cevap Verme" moduna al

def sor_bakalim(soru, input_text=""):
    # Soruyu şablona dök
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            soru,
            input_text,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    # Cevabı canlı olarak yazdır (Streaming)
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# --- Test Soruları ---
print("-" * 30)
print("SORU 1: Başkent Bilgisi")
sor_bakalim("Fransa'nın başkenti neresidir?")

print("-" * 30)
print("SORU 2: Yazılım Bilgisi")
sor_bakalim("Docker nedir?", "Yazılım Kavramları")

print("-" * 30)
print("SORU 3: Matematik Yeteneği")
sor_bakalim("25 ile 25 sayılarının toplamı kaçtır?", "Matematik işlemi")

print("-" * 30)
print("SORU 4: Zorlama Soru (Veride olmayan)")
sor_bakalim("Yapay zeka mühendisliği nedir?")

------------------------------
SORU 1: Başkent Bilgisi
<|begin_of_text|>Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
Fransa'nın başkenti neresidir?

### Girdi:


### Yanıt:
Fransa'nın başkenti Paris'dir.<|end_of_text|>
------------------------------
SORU 2: Yazılım Bilgisi
<|begin_of_text|>Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
Docker nedir?

### Girdi:
Yazılım Kavramları

### Yanıt:
Docker, bir yazılımı bir container içinde çalıştırmayı sağlayan bir platformdur.<|end_of_text|>
------------------------------
SORU 3: Matematik Yeteneği
<|begin_of_text|>Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
25 ile 25 sayılarının toplamı kaçtır?

### Girdi:
Matematik işlemi

### Yanıt:


In [20]:
# --- 6. ADIM: MODELİ KAYDETME (GGUF) ---
# Modeli 4-bit (q4_k_m) formatında sıkıştırıp kaydediyoruz.
# Bu işlem biraz RAM harcar, Colab bazen zorlanabilir ama genelde başarır.

model.save_pretrained_gguf("model_cikti", tokenizer, quantization_method = "q4_k_m")

print("✅ Model GGUF formatına çevrildi!")
print("Dosyalar sol taraftaki klasör simgesinde 'model_cikti' klasörü içinde.")

Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [10:38<00:00, 159.66s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [07:44<00:00, 116.16s/it]


Unsloth: Merge process complete. Saved to `/content/model_cikti`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3-8b.F16.gguf']
Unsloth: [2] Conv